## Preparació

In [10]:
# Paquets

import json, os, urllib, zipfile
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

In [11]:
# Paràmetres

# Directori de treball
WORK_DIR = os.path.abspath(os.getcwd())

# Enllaços dades
URL_PUNTS_ANC = 'https://opendata-ajuntament.barcelona.cat/data/dataset/6778b87d-da09-4eb8-9504-5c9ae5fbfad9/resource/6cea1468-f18e-438b-b1c5-ea7c0e287412/download'
URL_APARCAMENT = 'https://opendata-ajuntament.barcelona.cat/data/dataset/a7fd022c-d703-49e3-b628-7b32599a6a54/resource/85457026-c48d-4a95-ac9e-080756375d50/download'
URL_SERVEIS = 'https://opendata-ajuntament.barcelona.cat/data/dataset/12bb6230-1a79-4371-89c5-d59aaa67152e/resource/f7e37859-7f3c-4dcb-ac70-91eae481dd26/download'

URL_BICING_INFO = 'https://api.bsmsa.eu/ext/api/bsm/gbfs/v2/en/station_information'
URL_BICING_STATUS = 'https://api.bsmsa.eu/ext/api/bsm/gbfs/v2/en/station_status'


In [12]:
# Funcions

def get_data(url, filename):
    """Descarrega les dades a la carpeta 'datasets'."""
    try:
        urllib.request.urlretrieve(url, WORK_DIR + '/datasets/' + filename)
        print(filename + " descarregat correctament.")
    except:
        print("Problema descarregant: " + filename)
        
def save_data(gdf, filename):
    """Guarda un geodataframe en format SHP, el comprimeix i el desa a la carpeta 'map'."""
    gdf.to_file('tmp/' + filename + '.shp')
    zf = zipfile.ZipFile('map/' + filename + ".zip", "w", zipfile.ZIP_DEFLATED)
    zf.write('tmp/' + filename + ".dbf")
    zf.write('tmp/' + filename + ".shp")
    zf.write('tmp/' + filename + ".prj")
    zf.write('tmp/' + filename + ".shx")
    zf.write('tmp/' + filename + ".cpg")
    zf.close()
    print(filename + ' guardat correctament.')

In [13]:
# Districtes de Barcelona

districtes = gpd.read_file('datasets/DISTRICTES/DISTRICTES.shp')

## Punts d'ancoratge

In [14]:
# Lectura i parsejament de dades
get_data(URL_PUNTS_ANC, 'ancoratge.xml')
tree = ET.parse('datasets/ancoratge.xml')
root = tree.getroot()

rows = []
for row in root.findall('.//row'):
    item = row.find("item")
    
    # Address
    address = ''
    if ET.iselement(item.find("address")):
        address = item.find("address").text
    
    # Coords
    lat = item.find("gmapx").text
    lon = item.find("gmapy").text
        
    rows.append({'address':address, 'lat':lat, 'lon':lon})

# Creació geodataframe
punts_ancoratge = pd.DataFrame(rows)
punts_ancoratge = gpd.GeoDataFrame(punts_ancoratge,
                                   geometry = gpd.points_from_xy(punts_ancoratge.lon, punts_ancoratge.lat))

# Conversió del sistema de referència espacial
punts_ancoratge = punts_ancoratge.set_crs('EPSG:4326')
punts_ancoratge = punts_ancoratge.to_crs('EPSG:25831')

# Districte
punts_ancoratge = gpd.sjoin(punts_ancoratge, districtes, how='left', op='within')

# Atributs
punts_ancoratge.drop(columns=['index_right','lat','lon'], inplace=True)

punts_ancoratge.head()

ancoratge.xml descarregat correctament.


,address,geometry,districte
0,C Torrent de l'Olla 183,POINT (429230.906 4584116.954),Gràcia
1,C Riera de Sant Miquel 49,POINT (429417.054 4583327.384),Gràcia
2,C Vic 25,POINT (429339.489 4583426.846),Gràcia
3,Trav Gràcia 115,POINT (429314.611 4583423.647),Gràcia
4,C Luis Antúnez 1,POINT (429468.678 4583392.611),Gràcia


## Serveis

In [15]:
# Captura i parsejament de dades
get_data(URL_SERVEIS, 'serveis.xml')
tree = ET.parse('datasets/serveis.xml')
root = tree.getroot()

rows = []
for row in root.findall('.//row'):
    item = row.find("item")
    
    # Name
    name = item.find("name").text
    
    # Address
    address = ''
    if ET.iselement(item.find("address")):
        address = item.find("address").text
    
    # Coords
    lat = item.find("gmapx").text
    lon = item.find("gmapy").text
    
    # Web
    web = ''
    if ET.iselement(item.find("code_url")):
        web = item.find("code_url").text

    # Phone
    phone = ''
    if ET.iselement(item.find("phonenumber")):
        phone = item.find("phonenumber").text
        
    rows.append({'name':name, 'address':address, 'lat':lat, 'lon':lon, 'web': web, 'phone': phone})

# Creació geodataframe
serveis = pd.DataFrame(rows)
serveis = gpd.GeoDataFrame(serveis, geometry = gpd.points_from_xy(serveis.lon, serveis.lat))

# Conversió del sistema de referència espacial
serveis = serveis.set_crs('EPSG:4326')
serveis = serveis.to_crs('EPSG:25831')

# Districte
serveis = gpd.sjoin(serveis, districtes, how='left', op='within')

# Atributs
serveis.drop(columns=['lat','lon','index_right'], inplace = True)

serveis.head()

serveis.xml descarregat correctament.


,name,address,web,phone,geometry,districte
0,Pura Vida Rickshaw Tours,C Lafont 19,www.puravida.barcelona,931823930,POINT (430498.174 4580471.153),Sants-Montjuïc
1,Roda Lliure,C Consell de Cent 595,,673 475 560,POINT (431843.146 4584084.343),Sant Martí
2,BiciHub *Can Picó,C Pere IV 58,bicihub.barcelona,685 86 50 56,POINT (432467.921 4583013.617),Sant Martí
3,Barcelona Cycle Co,C Vigatans 2,,933 153 259,POINT (431409.566 4581713.648),Ciutat Vella
4,Bikes Barcelona rental,C Marina 138,www.bikesbarcelonarental.es,674 60 06 74,POINT (431792.812 4583181.378),Eixample


## Bicing

In [16]:
# Captura dades

# Informació de les estacions
with urllib.request.urlopen(URL_BICING_INFO) as url:
    data = json.loads(url.read().decode())

bicing_info = pd.DataFrame(data['data']['stations'])

# Informació de l'estat de les estacions
with urllib.request.urlopen(URL_BICING_STATUS) as url:
    data = json.loads(url.read().decode())

bicing_status = pd.DataFrame(data['data']['stations'])

In [17]:
# Unim dades d'informació i estat
bicing_stations = pd.merge(bicing_info, bicing_status, on='station_id', how='left')\
                    .rename(columns={'num_bikes_available':'bikes'})

# Bicis disponibles
bicing_stations['bikes_mec'] = bicing_stations['num_bikes_available_types'].apply(lambda x: x['mechanical'])
bicing_stations['bikes_elec'] = bicing_stations['num_bikes_available_types'].apply(lambda x: x['ebike'])

# Seleccionem columnes
COLUMNS = ['station_id','name','lat','lon','altitude','address','capacity',
           'bikes', 'bikes_mec', 'bikes_elec', 'status']
bicing_stations = bicing_stations[COLUMNS]

# Generem un geodataframe
bicing = gpd.GeoDataFrame(bicing_stations, geometry = gpd.points_from_xy(bicing_stations.lon, bicing_stations.lat))

# Conversió del sistema de referència espacial
bicing = bicing.set_crs('EPSG:4326')
bicing = bicing.to_crs('EPSG:25831')

# Districte
bicing = gpd.sjoin(bicing, districtes, how='left', op='within').drop(columns=['index_right'])

bicing.head()

,station_id,name,lat,lon,altitude,address,capacity,bikes,bikes_mec,bikes_elec,status,geometry,districte
0,1,"GRAN VIA CORTS CATALANES, 760",41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",46,4,4,0,IN_SERVICE,POINT (431461.865 4583262.211),Eixample
1,2,"C/ ROGER DE FLOR, 126",41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",27,2,2,0,IN_SERVICE,POINT (431216.112 4582988.058),Eixample
2,3,"C/ NÀPOLS, 82",41.394156,2.181331,11.0,"C/ NÀPOLS, 82",27,15,15,0,IN_SERVICE,POINT (431560.142 4582836.909),Eixample
3,4,"C/ RIBES, 13",41.393317,2.181248,8.0,"C/ RIBES, 13",21,5,5,0,IN_SERVICE,POINT (431552.390 4582743.896),Eixample
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",39,14,14,0,IN_SERVICE,POINT (431460.445 4582498.970),Ciutat Vella


## Aparcament

In [18]:
# Lectura i parsejament de dades
get_data(URL_APARCAMENT, 'aparcament.xml')
tree = ET.parse('datasets/aparcament.xml')
root = tree.getroot()

rows = []
for row in root.findall('.//row'):
    item = row.find("item")
    
    # Name
    name = item.find("name").text
    
    # Address
    address = ''
    if ET.iselement(item.find("address")):
        address = item.find("address").text
    
    # Coords
    lat = item.find("gmapx").text
    lon = item.find("gmapy").text
    
    # Web
    web = ''
    if ET.iselement(item.find("code_url")):
        web = item.find("code_url").text

    # Phone
    phone = ''
    if ET.iselement(item.find("phonenumber")):
        phone = item.find("phonenumber").text
        
    rows.append({'name':name, 'address':address, 'lat':lat, 'lon':lon, 'web': web, 'phone': phone})

# Creació geodataframe
aparcament = pd.DataFrame(rows)
aparcament = gpd.GeoDataFrame(aparcament,
                                   geometry = gpd.points_from_xy(aparcament.lon, aparcament.lat))

# Conversió del sistema de referència espacial
aparcament = aparcament.set_crs('EPSG:4326')
aparcament = aparcament.to_crs('EPSG:25831')

# Districte
aparcament = gpd.sjoin(aparcament, districtes, how='left', op='within')

# Atributs
aparcament.drop(columns=['index_right','lat','lon'], inplace=True)

aparcament.head(10)

aparcament.xml descarregat correctament.


,name,address,web,phone,geometry,districte
0,Aparcament Travessera de Dalt - Park Güell,Trav Dalt 51,www.aparcamentsbsm.cat,937608077,POINT (429321.830 4584553.634),Gràcia
1,Aparcament BAMSA València - Calàbria,C València 77,www.saba.es,902283080,POINT (429028.594 4581693.721),Eixample
2,Aparcament Gràcia Motos,C Gran de Gràcia 190,www.aparcamentsbsm.cat,934155753,POINT (429144.720 4583866.538),Gràcia
3,Aparcament Plaça Joanic,C Escorial 1,www.aparcamentsbsm.cat,932846345,POINT (430006.997 4584133.948),Gràcia
4,Aparcament BAMSA Londres-Villarroel,C Londres 56,www.saba.es,934108136,POINT (428762.953 4582533.711),Eixample
5,Aparcament Concepción Arenal,C Concepción Arenal 156,www.aparcamentsbsm.cat,933406095,POINT (431800.365 4586345.515),Sant Andreu
6,Aparcament Plaça del Fòrum,Pg Taulat 278,www.aparcamentsbsm.cat,933561199,POINT (434913.411 4584885.359),Sant Martí
7,Aparcament Cotxeres de Sarria,Pg Manuel Girona 77,www.aparcamentsbsm.cat,932802740,POINT (427195.651 4582695.408),Sarrià-Sant Gervasi
8,Aparcament Mercat Sagrada Família,C Mallorca 425,www.aparcamentsbsm.cat,934365155,POINT (431233.415 4584046.807),Eixample
9,Aparcament BAMSA Vilardell,C Vilardell 20,www.saba.es,902283080,POINT (428309.992 4580695.980),Sants-Montjuïc


## Exportació

In [19]:
# Exportació dades

save_data(punts_ancoratge, 'PUNTS_ANCORATGE')
save_data(bicing, 'BICING')
save_data(serveis, 'SERVEIS')
save_data(aparcament, 'APARCAMENT')

PUNTS_ANCORATGE guardat correctament.
BICING guardat correctament.
SERVEIS guardat correctament.
APARCAMENT guardat correctament.


In [57]:
# Guardem l'identificador de les estacions per a actualitzacions

bicing_info = bicing_info.loc[:,['station_id']]
bicing_info.to_csv('datasets/bicing_info.csv', index=False)